In [ ]:
SYFT_VERSION = ">=0.8.1b0,<=0.9"
package_string = f'"syft{SYFT_VERSION}"'
%pip install {package_string} -f https://whls.blob.core.windows.net/unstable/index.html -q

In [ ]:
import syft as sy
sy.requires(SYFT_VERSION)
from syft import autocache

In [ ]:
node = sy.orchestra.launch(name="test-domain-1", port="auto", dev_mode=True, reset=True)

In [ ]:
domain_client = node.login(email="info@openmined.org", password="changethis")

In [ ]:
domain_client.api

In [ ]:
data_subjects = domain_client.api.services.data_subject.get_all()

In [ ]:
data_subjects

In [ ]:
assert len(data_subjects) == 0

In [ ]:
country = sy.DataSubject(name="Country", aliases=["country_code"])

In [ ]:
canada = sy.DataSubject(name="Canada", aliases=["country_code:ca"])
germany = sy.DataSubject(name="Germany", aliases=["country_code:de"])
spain = sy.DataSubject(name="Spain", aliases=["country_code:es"])
france = sy.DataSubject(name="France", aliases=["country_code:fr"])
japan = sy.DataSubject(name="Japan", aliases=["country_code:jp"])
uk = sy.DataSubject(name="United Kingdom", aliases=["country_code:uk"])
usa = sy.DataSubject(name="United States of America", aliases=["country_code:us"])
australia = sy.DataSubject(name="Australia", aliases=["country_code:au"])
india = sy.DataSubject(name="India", aliases=["country_code:in"])

In [ ]:
country.add_member(canada)
country.add_member(germany)
country.add_member(spain)
country.add_member(france)
country.add_member(japan)
country.add_member(uk)
country.add_member(usa)
country.add_member(australia)
country.add_member(india)

country.members

In [ ]:
registry = domain_client.data_subject_registry

In [ ]:
response = registry.add_data_subject(country)

In [ ]:
response

In [ ]:
assert response

In [ ]:
domain_client.data_subject_registry

In [ ]:
data_subjects = domain_client.api.services.data_subject.get_all()

In [ ]:
assert len(data_subjects) == 10

In [ ]:
canada_dataset_url = "https://github.com/OpenMined/datasets/blob/main/trade_flow/ca%20-%20feb%202021.csv?raw=True"

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(autocache(canada_dataset_url))

In [ ]:
df

In [ ]:
ca_data = df[0:10]

In [ ]:
mock_ca_data = df[10:20]

In [ ]:
ca_data

In [ ]:
dataset = sy.Dataset(name="Canada Trade Value")

In [ ]:
dataset.set_description("""Canada Trade Data""")

In [ ]:
dataset.add_citation("Person, place or thing")
dataset.add_url("https://github.com/OpenMined/datasets/tree/main/trade_flow")

In [ ]:
dataset.add_contributor(role=sy.roles.UPLOADER, 
                                name="Andrew Trask", 
                                email="andrew@openmined.org",
                                note="Andrew runs this domain and prepared the dataset metadata.")

dataset.add_contributor(role=sy.roles.EDITOR, 
                                name="Madhava Jay", 
                                email="madhava@openmined.org",
                                note="Madhava tweaked the description to add the URL because Andrew forgot.")

In [ ]:
dataset.contributors

In [ ]:
assert len(dataset.contributors) == 2

In [ ]:
ctf = sy.Asset(name="canada_trade_flow")
ctf.set_description("""all the datas""")

In [ ]:
ctf.add_contributor(role=sy.roles.UPLOADER, 
                      name="Andrew Trask", 
                      email="andrew@openmined.org",
                      note="Andrew runs this domain and prepared the asset.")

In [ ]:
ctf.set_obj(ca_data)

In [ ]:
ctf.set_shape((10, 22))

In [ ]:
ctf.add_data_subject(canada)

In [ ]:
ctf.no_mock()
dataset.add_asset(ctf)

In [ ]:
dataset.remove_asset(name=ctf.name)

In [ ]:
ctf.set_mock(mock_ca_data, mock_is_real=False)

In [ ]:
dataset.add_asset(ctf)

In [ ]:
domain_client.upload_dataset(dataset)

In [ ]:
datasets = domain_client.api.services.dataset.get_all()

In [ ]:
assert len(datasets) == 1

In [ ]:
datasets

In [ ]:
mock = domain_client.datasets[0].assets[0].mock

In [ ]:
assert mock_ca_data.equals(mock)

In [ ]:
real = domain_client.datasets[0].assets[0].data

In [ ]:
assert ca_data.equals(real)

In [ ]:
domain_client.register(name="Jane Doe", email="jane@caltech.edu", password="abc123", institution="Caltech", website="https://www.caltech.edu/")

In [ ]:
if node.node_type.value == "python":
    node.land()